In [1]:
# activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [52.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ge

In [3]:
#import packages

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType

# we are going to use this to time our queries.
import time

# Create a SparkSession
spark = SparkSession.builder.appName("PySpark").getOrCreate()
from pyspark import SparkFiles

In [9]:
pp_df = spark.read.text('peterpan.txt')


+--------------------+
|               value|
+--------------------+
|All children, exc...|
|Of course they li...|
|The way Mr. Darli...|
|Mr. Darling used ...|
|Mrs. Darling was ...|
|Wendy came first,...|
|For a week or two...|
|“Now don't interr...|
|“I have one pound...|
|“Of course we can...|
|“Remember mumps,”...|
|There was the sam...|
|Mrs. Darling love...|
|No nursery could ...|
|He had his positi...|
|Nana also trouble...|
|Mrs. Darling firs...|
|I don't know whet...|
|Of course the Nev...|
|Of all delectable...|
+--------------------+
only showing top 20 rows



In [39]:
pp_data = spark.sparkContext.textFile('peterpan.txt')
pp_data_split = pp_data.flatMap(lambda line: line.split(" "))

In [40]:
pp_data_cnt = pp_data_split.map(lambda word: (word, 1))

In [41]:
from operator import add
pp_data_cnt_all = pp_data_cnt.reduceByKey(add)

In [43]:
pp_data_cnt_all.saveAsTextFile('wordCnts.txt')

In [44]:
spark.read.text('wordCnts.txt').show()

+--------------+
|         value|
+--------------+
|  ('one,', 15)|
| ('They', 101)|
|  ('know', 64)|
|   ('way', 58)|
|  ('was', 897)|
|    ('One', 9)|
| ('when', 151)|
|   ('two', 36)|
|  ('years', 3)|
|   ('in', 623)|
|('garden,', 1)|
|('plucked', 2)|
| ('flower', 2)|
|   ('ran', 17)|
|  ('her', 361)|
|('suppose', 7)|
|  ('must', 59)|
| ('have', 243)|
|('looked', 33)|
|('rather', 40)|
+--------------+
only showing top 20 rows

